In [1]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id = tokenizer.eos_token_id)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


### Greedy Search

In [3]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors = 'tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length = 50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll


### Beam search

In [4]:
beam_output = model.generate(
    input_ids,  
    max_length = 50, 
    num_beams = 5, 
    early_stopping = True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure if I'll


In [5]:
# set no_repeat_ngram_size to 2 as repetition penalty
beam_output = model.generate(
    input_ids, 
    max_length = 50, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    early_stopping = True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break


In [6]:
# set return_num_sequences > 1, assertion(num_return_sequences <= num_beams) as num_beams are all probs shown so sequences returned cannot be greater than all possible probs
beam_outputs = model.generate(
    input_ids, 
    max_length = 50, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens = True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to get back to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to take a break
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to get back to
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about 

### Sampling

In [7]:
tf.random.set_seed(0)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample = True, 
    max_length = 50, 
    top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog because of myself.

Bryan: Oh, it's edgy.

Rachel: Does your dog love weddings?

Bryan: He won't, because he wants to

Rachel:


In [8]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample = True, 
    max_length = 50, 
    top_k = 0, 
    temperature = 0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog at the zoo, so I'm going to go out on a limb and say that I absolutely hate standing on my dog's back. But if you're not feeling just fine, it's an important first step toward


### Top-K Sampling

In [9]:
# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample = True, 
    max_length = 50, 
    top_k = 50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog and getting my water fountain ready, so I'm getting my dog in here about 11:00 am. We go to find some coffee, a cup of tea, a little cup of chocolate, some chocolate popcorn and


### Top-p (nucleus) sampling

In [10]:
# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample = True, 
    max_length = 50, 
    top_p = 0.92, 
    top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog and I do love dancing. But I am not trying to kill anybody. I just want to get my loving dog back and I try hard to turn off him. He breaks me off. I want to protect my


In [11]:
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog and eating something amazing that I like. When they got home the first time they asked me where I would eat and I told them what I used to do back home. They said they were all really happy with their
1: I enjoy walking with my cute dog and have a great time here at Wanda in downtown San Antonio."

He added that it was a nice and quiet neighborhood that felt more welcoming than some of the other neighborhoods in the area.

"
2: I enjoy walking with my cute dog, I like taking pictures, playing and I love to talk about them," she said.

They also like to do things together, saying she enjoyed having her dog play outdoors at her home in Vancouver and wanted
